In [15]:
import pandas as pd
import numpy as np
import networkx as nx
import xgi
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import torch.nn as nn

In [25]:
embeddings = pd.read_csv('full_users_embeddings.csv')
embeddings = embeddings.iloc[:, np.r_[0:3, -768:0]]
embeddings['one_hot'] = 0

df = pd.read_json('babynamesDB_users.json')
hyperedge_list = df[['_id', 'groups']]

# only keep users that are in the embeddings dataframe


In [49]:
hyperedge_list_copy = hyperedge_list.copy()
hyperedge_list_copy = hyperedge_list_copy[hyperedge_list_copy['_id'].isin(embeddings['_id'].values)]
hyperedge_list_copy['_id'] = hyperedge_list_copy.index.astype(int)

hyperedge_list_copy = hyperedge_list_copy.explode('groups')
# turn groups to one hot encoding
hyperedge_list_copy['groups'] = hyperedge_list_copy['groups'].astype('category')

hyperedge_list_copy['groups'] = hyperedge_list_copy['groups'].cat.codes.astype(int)


H = xgi.from_bipartite_pandas_dataframe(hyperedge_list_copy)



In [57]:
G = xgi.to_bipartite_graph(H)

In [62]:
num_groups = H.num_edges
num_embeddings = embeddings.shape[1]
column_names = embeddings.columns
new_rows = []
for i in range(1, num_groups + 1):
    #create a new row for each group
    new_row = pd.DataFrame(np.zeros((1, num_embeddings)), columns = column_names)
    new_row['one_hot'] = i
    new_rows.append(new_row)


embeddings = pd.concat([embeddings] + new_rows, ignore_index=True)
embeddings = embeddings.iloc[:-14, :]


In [68]:
#add embeddings as node features, each row is its corresponding node
nx.set_node_attributes(G, (embeddings.iloc[:, 3:].to_dict('index')))

In [ ]:
def graph_to_edge_list(G):
  return list(G.edges())

def edge_list_to_tensor(edge_list):
  return torch.tensor(edge_list).t()

pos_edge_list = graph_to_edge_list(G)

def sample_negative_edges(G, num_neg_samples):
  possible_edges = [(u,v) for u in G.nodes() for v in G.nodes() if u != v and (u,v) not in G.edges()]
  neg_edge_list = random.sample(possible_edges, num_neg_samples)
  return np.array(neg_edge_list)